In [1]:
!pip install transformers

In [2]:
import pandas as pd
import numpy as np
import transformers as ppb
import torch
import warnings
warnings.filterwarnings('ignore')

In [4]:
df=pd.read_csv("Final data.csv")

In [5]:
df

,Unnamed: 0,Title,Post Text,Category
0,0,Testing requirements in toys and games category,My product is domino sets for adults. I am the...,Amazon Custom
1,1,Food and Drug Administration,"Hello, I am on my way to becoming an Amazon de...",Amazon Custom
2,2,Images Storage for mass upload,When uploading products with the spreadsheet w...,Amazon Custom
3,3,Prepaid Return Label program email clarification,You may have received an email from Amazon inf...,Amazon Custom
4,4,Pulling Amazon Custom Personalizatoin Data fro...,The problem: Currently Amazon gives you links ...,Amazon Custom
...,...,...,...,...
1810,1810,A suspended account which 20% from his profits...,20% of the profits are for charity. Can I expe...,Account Health
1811,1811,Need help with account suspension Appeal,"Hello members,\nI am requesting for some advic...",Account Health
1812,1812,Restricted Product Policy Violations,So I am a new seller and just noticed that i h...,Account Health
1813,1813,8k on hold - sourcing from Home Depot - inauth...,"Hi,\nIm a new seller on Amazon and have been s...",Account Health


In [6]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-cased')

# For BERT: 
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [ ]:
tokenized = df.loc[:,'Title'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
      max_len = len(i)
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
  last_hidden_states = model(input_ids, attention_mask=attention_mask)
features1 = last_hidden_states[0][:,0,:].numpy()


In [ ]:
pd.DataFrame(features1).to_csv("bert_processed_titles.csv")
from google.colab import files
files.download("bert_processed_titles.csv")

In [ ]:
tokenized = df.loc[:,'Post Text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length=150,truncation=True)))
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
      max_len = len(i)
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
  last_hidden_states = model(input_ids, attention_mask=attention_mask)
features1 = last_hidden_states[0][:,0,:].numpy()

In [ ]:
pd.DataFrame(features1).to_csv("bert_processed_post.csv")
from google.colab import files
files.download("bert_processed_post.csv")